# 課程目標:
了解Keras 函數式 API 的使用 <br />
# 範例重點:
了解函數式 API 的彈性

In [1]:
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model

#主要輸入接收新聞標題本身，即一個整數序列（每個整數編碼一個詞）。
#這些整數在1 到10,000 之間（10,000 個詞的詞彙表），且序列長度為100 個詞
#宣告一個 NAME 去定義Input
main_input = Input(shape = (100,), dtype = 'int32', name = 'main_input')

# Embedding 層將輸入序列編碼為一個稠密向量的序列，
# 每個向量維度為 512。
x = Embedding(output_dim = 512, input_dim = 10000, input_length = 100)(main_input)

# LSTM 層把向量序列轉換成單個向量，
# 它包含整個序列的上下文信息
lstm_out = LSTM(32)(x)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [2]:
#插入輔助損失，使得即使在模型主損失很高的情況下，LSTM 層和Embedding 層都能被平穩地訓練
auxiliary_output = Dense(1, activation = 'sigmoid', name = 'aux_output')(lstm_out)

In [3]:
#輔助輸入數據與LSTM 層的輸出連接起來，輸入到模型
import keras
auxiliary_input = Input(shape = (5,), name = 'aux_input')
x = keras.layers.concatenate([lstm_out, auxiliary_input])

# 堆疊多個全連接網路層
x = Dense(64, activation = 'relu')(x)
x = Dense(64, activation = 'relu')(x)
#作業解答: 新增兩層
x = Dense(64, activation = 'relu')(x)
x = Dense(64, activation = 'relu')(x)

# 最後添加主要的邏輯回歸層
main_output = Dense(1, activation = 'sigmoid', name = 'main_output')(x)

In [4]:
# 宣告 MODEL API, 分別採用自行定義的 Input/Output Layer
model = Model(inputs = [main_input, auxiliary_input], outputs = [main_output, auxiliary_output])

In [5]:
model.compile(optimizer = 'rmsprop',
             loss = {'main_output' : 'binary_crossentropy', 'aux_output' : 'binary_crossentropy'},
             loss_weights = {'main_output' : 1. , 'aux_output' : 0.2})

In [6]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 512)     5120000     main_input[0][0]                 
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 32)           69760       embedding_1[0][0]                
__________________________________________________________________________________________________
aux_input (InputLayer)          (None, 5)            0                                            
__________________________________________________________________________________________________
concatenat

# 作業目標:
建立一個網路模型 <br />
# 作業重點:
請修改 Name 中, 自定義的 Layer 名稱 <br />
<br />
增加一層全連階層 <br />
<br />
宣告 MODEL API, 分別採用自行定義的 Input/Output Layer <br />
<br />
model.summary 查看 Layers stack

In [9]:
from keras import datasets
from keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Flatten


In [29]:
import numpy as np

main_input = Input( shape = (28,28,1), dtype = 'float32', name = 'main_input')
x = Conv2D(filters = 32, kernel_size = (3,3), padding = 'same', activation = 'relu')(main_input)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Flatten()(x)
out = Dense(256, activation = 'relu')(x)
first_output = Dense(10, activation = 'softmax', name = 'first_output')(out)

second_Input = Input( shape = (784, ), dtype = 'float32', name = 'second_input')
x = keras.layers.concatenate([out, second_Input])
x = Dense(256, activation = 'relu')(x)
x = Dense(128, activation = 'relu')(x)
x = Dense(64, activation = 'relu')(x)
main_output = Dense(10, activation = 'softmax', name = 'main_output')(x)

In [30]:
model = Model(inputs = [main_input, second_Input], outputs = [first_output, main_output])

model.compile(optimizer = 'rmsprop',
             loss = {'main_output' : 'categorical_crossentropy', 'first_output' : 'categorical_crossentropy'},
             loss_weights = {'main_output' : 0.3 , 'first_output' : 1.},
             metrics = {'main_output' : 'accuracy', 'first_output' : 'accuracy'})

In [31]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d_14 (Conv2D)              (None, 28, 28, 32)   320         main_input[0][0]                 
__________________________________________________________________________________________________
max_pooling2d_10 (MaxPooling2D) (None, 14, 14, 32)   0           conv2d_14[0][0]                  
__________________________________________________________________________________________________
flatten_7 (Flatten)             (None, 6272)         0           max_pooling2d_10[0][0]           
__________________________________________________________________________________________________
dense_29 (